In [1]:
from rag import RetrievalQA
from retriever import Retriever, BM25_Retriever
import pandas as pd
from datasets import Dataset

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# retriever 

In [2]:
retriever = Retriever()
# retriever = BM25_Retriever()


/home/ubuntu/legal_qa/legal_rag/RAG/retriever.py:45: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# HyDE 


In [3]:
from hyde.generator import OpenAIGenerator
from hyde.promptor import Promptor
from hyde.hyde import HyDE

In [ ]:
model_name = "gpt-3.5-turbo"
api_key = ""

In [5]:
generator = OpenAIGenerator(model_name=model_name, api_key=api_key)
promptor = Promptor()
hyde = HyDE(promptor, generator,retriever.encoder,retriever)

In [6]:
question = "배우자에게 폭행을 당했습니다 이때 상대방의 귀책사유로 이혼을 요구할 수 있나요?"

In [7]:
retrieved_docs = hyde.search(question,k=3)

In [8]:
retrieved_docs[0].metadata['summary']

'민법 제840조 제3호 소정의 이혼사유인 ‘배우자로부터 심히 부당한 대우를 받았을 때’라고 함은 혼인 당사자의 일방이 배우자로부터 혼인관계의 지속을 강요하는 것이 가혹하다고 여겨질 정도의 폭행이나 학대 또는 중대한 모욕을 받았을 경우를 말하는 것이다(대법원 1999. 2. 12. 선고 97므612 판결 등 참조).\n살피건대, 앞서 든 증거들 및 변론 전체의 취지에 의하면, 혼인기간 중 원고가 피고를 배려하려고 노력하는데도 피고가 원고에게 자주 짜증을 내거나 화를 내고 불만을 표현하였던 것은 사실이나, 이는 피고가 혼인하면서 직장을 그만두고 주거지도 옮긴데다가 혼인한 지 얼마 되지 않아 사건본인을 임신, 출산하고 양육하는 과정에서 정신적·육체적으로 힘든 상황에 놓여 있어, 남편인 원고에게 힘든 감정을 쏟아내게 되었던 것으로 보이는 점, 피고는 자신의 감정과 욕구를 적극적으로 표현하는 성격인 반면 원고는 상대방에게 맞춰주고 참는 성격인데, 피고가 원고의 성격에 따른 특성을 잘 고려하지 못한 채 위와 같이 행동했던 점, 원고가 피고에게 이혼을 요구하자 피고가 자신의 잘못을 인식하고 사과하였던 점 등에 비추어 보면, 원고가 제출한 증거들만으로는 원고가 혼인기간 중 피고로부터 혼인관계의 지속을 강요하는 것이 가혹하다고 여겨질 정도의 폭행이나 학대 또는 중대한 모욕을 받았음을 인정하기에 부족하고, 달리 이를 인정할 증거가 없다.'

# Reranking

In [9]:
from rerank.reranker import Reranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [10]:
reranker = HuggingFaceCrossEncoder(model_name="Dongjin-kr/ko-reranker")
rerank_retriever = Reranker(retriever,reranker, top_k=2)

In [11]:
reranked_docs = rerank_retriever.search(question)

In [12]:
reranked_docs[0].metadata["summary"]

'원고와 피고 사이의 혼인관계는 더 이상 회복할 수 없을 정도로 파탄되고, 이는 피고가 원고 및 원고의 아버지를 형사고소하고, 원고가 무면허 운전 등으로 형사재판을 받게 되자 원고의 구속 및 엄벌을 바라는 취지의 탄원서를 2회 제출하였으며, 혜화동 부동산을 처분하고도 명륜동 부동산과 관련된 원고의 채무는 원고의 잘못 때문에 발생하였다며 이를 전혀 변제하지 아니하여 현재까지 원고가 7억 원 이상의 부채로 인해 신용불량자 상태에서 벗어나지 못하고 있는 등 혼인관계를 파국으로 치닫게 한 잘못과, 원고가 다른 여자를 만나 이성교제를 하고, 경제적인 어려움이 닥치자 이를 모두 피고의 탓으로 돌리면서 피고를 폭행하는 등 부당한 대우를 한 잘못이 경합되어 초래되었다고 할 것인바, 혼인생활 파탄에 있어 원고와 피고의 책임은 상호 대등한 정도라고 보여지고, 피고의 위와 같은 잘못은 민법 제840조 제3호, 제6호에 정한 재판상 이혼사유에 해당하므로, 원고의 이혼청구는 이유 있다고 판단해 상고를 기각한다.'

# Recomp

In [13]:
from recomp.recomp import Recomp
from openai import OpenAI

In [14]:
client = OpenAI(api_key = api_key )

In [15]:
recomp_retriever = Recomp(generator = client, retriever = retriever)

In [16]:
compressed_docs = recomp_retriever.search(question,k = 2)

In [17]:
compressed_docs[0].metadata["summary"]

'피고가 폭행이나 학대를 가한 것으로 인정되지 않아, 상대방의 귀책사유로 이혼을 요구할 수 없다고 판결되었습니다.'

# QA 시스템

In [18]:
from rag import RetrievalQA

In [19]:
qa = RetrievalQA(model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B",retriever = rerank_retriever,top_k = 2)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.50s/it]


In [20]:
rerank_output = qa.generate(question)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[Document(metadata={'case_title': '대법원 2004. 2. 27. 선고 2003므1890 판결', 'summary': '원고와 피고 사이의 혼인관계는 더 이상 회복할 수 없을 정도로 파탄되고, 이는 피고가 원고 및 원고의 아버지를 형사고소하고, 원고가 무면허 운전 등으로 형사재판을 받게 되자 원고의 구속 및 엄벌을 바라는 취지의 탄원서를 2회 제출하였으며, 혜화동 부동산을 처분하고도 명륜동 부동산과 관련된 원고의 채무는 원고의 잘못 때문에 발생하였다며 이를 전혀 변제하지 아니하여 현재까지 원고가 7억 원 이상의 부채로 인해 신용불량자 상태에서 벗어나지 못하고 있는 등 혼인관계를 파국으로 치닫게 한 잘못과, 원고가 다른 여자를 만나 이성교제를 하고, 경제적인 어려움이 닥치자 이를 모두 피고의 탓으로 돌리면서 피고를 폭행하는 등 부당한 대우를 한 잘못이 경합되어 초래되었다고 할 것인바, 혼인생활 파탄에 있어 원고와 피고의 책임은 상호 대등한 정도라고 보여지고, 피고의 위와 같은 잘못은 민법 제840조 제3호, 제6호에 정한 재판상 이혼사유에 해당하므로, 원고의 이혼청구는 이유 있다고 판단해 상고를 기각한다.'}, page_content='인관계의 지속을 강요하는 것이 참으로 가혹하다고 여겨질 정도의 폭행이나 학대 또는 모욕을 받았을 경우 이혼의 사유로 배우자로부터 심히 부당한 대우를 받았을 때라고 볼 수 있는가?'), Document(metadata={'case_title': '부산가정법원 2018. 8. 23. 선고 2017드합200439 판결', 'summary': '피고가 폭행이나 학대를 가한 것으로 인정되지 않아, 상대방의 귀책사유로 이혼을 요구할 수 없다고 판결되었습니다.'}, page_content='민법 제840조 제3호 소정의 이혼사유인 ‘배우자로부터 심히 부당한 대우를 받았을 때’라고 함은 혼인 당사자의 일방이 배우자로부터 혼인관계의 지속을 강요하는 것이 가혹하다고 여겨질 정도의 폭행이나 학대 또는 중대한 모욕을 받았을 

In [21]:
rerank_output

'배우자에게 폭행을 당했습니다 이때 상대방의 귀책사유로 이혼을 요구할 수는 없습니다. 피고가 폭행이나 학대를 가한 것으로 인정되지 않으면, 상대방의 귀책사유로 이혼을 요구할 수 없습니다. (부산가정법원 2018. 8. 23. 선고 2017드합200439 판결 참조)'

In [23]:
qa.retriever = recomp_retriever
qa.top_k = 5

In [24]:
recomp_output = qa.generate(question)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[Document(metadata={'case_title': '부산가정법원 2018. 8. 23. 선고 2017드합200439 판결', 'summary': '피고가 폭행이나 학대를 가한 것으로 인정되지 않아, 상대방의 귀책사유로 이혼을 요구할 수 없다는 판결이 있습니다.'}, page_content='민법 제840조 제3호 소정의 이혼사유인 ‘배우자로부터 심히 부당한 대우를 받았을 때’라고 함은 혼인 당사자의 일방이 배우자로부터 혼인관계의 지속을 강요하는 것이 가혹하다고 여겨질 정도의 폭행이나 학대 또는 중대한 모욕을 받았을 경우를 말하는 것인가?'), Document(metadata={'case_title': '대구가정법원 2014. 6. 11. 선고 2013드단19474 판결', 'summary': '피고의 폭행으로 인한 이혼 요구는 증거 부족으로 인정되지 않음.'}, page_content='민법 제840조 제3호에서 정한 이혼사유인 ‘배우자로부터 심히 부당한 대우를 받았을 때’라 함은 혼인 당사자의 한쪽이 배우자부터 혼인관계의 지속을 강요받는 것이 가혹하다고 여겨질 정도의 폭행이나 학대 또는 중대한 모욕을 받았을 경우를 말하는가?'), Document(metadata={'case_title': '대법원 1990. 9. 25. 선고 89므112 판결', 'summary': '이혼을 요구할 수 없습니다. 상대방의 귀책사유로 혼인을 파탄시켰다고 해도 특별한 사정이 없는 한 이혼을 청구할 수 없습니다.'}, page_content='혼인을 파탄에 이르게한 사유에 관하여 당초 책임있는 배우자는 민법 제840조 제1항 제6호의 사유가 있다 하여 이혼을 청구할 수 있는가?'), Document(metadata={'case_title': '부산가정법원 2017. 7. 12. 선고 2017드단779 판결', 'summary': '피고가 원고를 수차례 폭행하고 회복 불가능한 정도까지 파탄이 된 경우, 귀책사유로 이혼을 요구할 수 있다.'}, page_content='혼인

In [25]:
recomp_output

'배우자에게 폭행을 당했을 때 상대방의 귀책사유로 이혼을 요구하기 위해서는 피고가 폭행이나 학대를 가한 것으로 인정되어야 합니다. 피고의 귀책사유가 인정된 경우에만 이혼을 요구할 수 있습니다. 반대로 귀책사유가 인정되지 않는 경우에는 이혼을 요구할 수 없습니다. \n\n예를 들어, 부산가정법원 2018. 8. 23. 선고 2017드합200439 판결과 대구가정법원 2014. 6. 11. 선고 2013드단19474 판결에서는 피고의 귀책사유가 인정되지 않아 이혼을 요구할 수 없다고 판결했습니다. 반면, 부산가정법원 2017. 7. 12. 선고 2017드단779 판결에서는 피고가 원고를 수차례 폭행하고 회복 불가능한 정도까지 파탄이 된 경우 귀책사유로 이혼을 요구할 수 있다고 판결했습니다.'

In [26]:
qa.retriever = hyde
qa.top_k = 2

In [27]:
hyde_output = qa.generate(question)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[Document(metadata={'case_title': '부산가정법원 2018. 8. 23. 선고 2017드합200439 판결', 'summary': '피고가 폭행이나 학대를 가한 것으로 인정되지 않아, 상대방의 귀책사유로 이혼을 요구할 수 없다는 판결이 있습니다.'}, page_content='민법 제840조 제3호 소정의 이혼사유인 ‘배우자로부터 심히 부당한 대우를 받았을 때’라고 함은 혼인 당사자의 일방이 배우자로부터 혼인관계의 지속을 강요하는 것이 가혹하다고 여겨질 정도의 폭행이나 학대 또는 중대한 모욕을 받았을 경우를 말하는 것인가?'), Document(metadata={'case_title': '대구가정법원 2014. 6. 11. 선고 2013드단19474 판결', 'summary': '피고의 폭행으로 인한 이혼 요구는 증거 부족으로 인정되지 않음.'}, page_content='민법 제840조 제3호에서 정한 이혼사유인 ‘배우자로부터 심히 부당한 대우를 받았을 때’라 함은 혼인 당사자의 한쪽이 배우자부터 혼인관계의 지속을 강요받는 것이 가혹하다고 여겨질 정도의 폭행이나 학대 또는 중대한 모욕을 받았을 경우를 말하는가?')]


In [28]:
hyde_output

'배우자에게 폭행을 당했더라도 상대방의 귀책사유로 이혼을 요구하기 위해서는 폭행이나 학대를 입증해야 합니다. 만약 폭행이나 학대를 입증할 수 없는 경우, 상대방의 귀책사유로 이혼을 요구할 수 없습니다.'